In [1]:
import numpy as np
import random
import time


In [2]:
def dibujar_batalla_barcos():
    escena = [
        "      __|__                     __|__      ",
        "     |o o o|                   |o o o|     ",
        "  ___/_____/\\__             ___/_____/\\__   ",
        "  \\           /             \\           /   ",
        "~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~",
        "               #-------->>                    ",
        "                BOOOOOM!                      ",]
    for linea in escena:
        print(linea)

In [3]:
def crear_tablero(tamaño):
    tablero = np.full((tamaño, tamaño), "_")
    return tablero

In [4]:
def crear_barco(eslora, tablero):
    while True:
        casilla_0 = (random.randint(0, 9), random.randint(0, 9))
        orientacion = random.choice(["Vertical", "Horizontal"])

        barco = [casilla_0]
        casilla = casilla_0

        for _ in range(1, eslora):
            if orientacion == "Vertical":
                nueva_casilla = (casilla[0] + 1, casilla[1])
            else:
                nueva_casilla = (casilla[0], casilla[1] + 1)

            if nueva_casilla[0] > 9 or nueva_casilla[1] > 9:
                break  
            barco.append(nueva_casilla)
            casilla = nueva_casilla

        if len(barco) == eslora:
            if not comprobar_colision(barco, tablero):
                return barco

In [5]:
def comprobar_colision(barco, tablero): 
    for i, j in barco:
        if tablero[i][j] == 'O':
            return True
    return False

In [6]:
def crear_flota(tablero, esloras):
    return [crear_barco(eslora, tablero) for eslora in esloras]

In [7]:
def colocar_flota(barcos, tablero): 
    for barco in barcos:
        for i, j in barco:
            tablero[i][j] = 'O'  
    return tablero

In [8]:
def generar_nuevo_tablero():
    tablero_usuario = crear_tablero(10)
    flota_usuario = crear_flota(tablero_usuario, [2, 2, 2, 3, 3, 4])
    tablero_final = colocar_flota(flota_usuario, tablero_usuario)
    return tablero_final

In [9]:
def sistema_de_turnos(tablero_usuario, tablero_maquina, flota_usuario, flota_maquina, modo):
    print("Que comience el juego")
    barcos_hundidos_usuario = 0
    barcos_hundidos_maquina = 0

    limite_puntos = 2 if modo == "corto" else len(tablero_maquina[tablero_maquina == "O"])

    while barcos_hundidos_usuario < limite_puntos and barcos_hundidos_maquina < limite_puntos:
        print("\n--- Tu turno ---")
        barcos_hundidos_maquina += turno_usuario(tablero_maquina, flota_maquina)
        mostrar_tablero(tablero_maquina)

        if barcos_hundidos_maquina >= limite_puntos:
            print("Has ganado makena.")
            break

        time.sleep(1)

        print("\n--- Turno de la Máquina ---")
        barcos_hundidos_usuario += turno_maquina(tablero_usuario, flota_usuario)
        mostrar_tablero(tablero_usuario)

        if barcos_hundidos_usuario >= limite_puntos:
            print("La máquina te ha ganado crack")
            break
        
        # Mostrar el conteo de barcos hundidos
        print(f"Barcos hundidos - Tú: {barcos_hundidos_maquina}, Máquina: {barcos_hundidos_usuario}")


In [10]:
def turno_usuario(tablero_maquina, flota_maquina):
    while True:
        try:
            fila = int(input("Introduce la fila (1-10) a la que quieres disparar: ")) - 1
            columna = int(input("Introduce la columna (1-10) a la que quieres disparar: ")) - 1
            if not (0 <= fila <= 9) or not (0 <= columna <= 9):
                print("Coordenadas fuera de rango, inténtelo de nuevo")
                continue
        except ValueError:
            print("Introduzca un número entre 1 y 10.")
            continue

        resultado = disparar((fila, columna), tablero_maquina)
        print(f"Disparaste a {(fila + 1, columna + 1)}:\n {resultado}")

        if resultado == "Agua":
            print("Fallaste. Fin de tu turno.")
            return 0
        elif resultado == "Tocado":
            print("Has acertado, sigue disparando.")
            for barco in flota_maquina:
                if barco_hundido(barco, tablero_maquina):
                    print("Barco hundido")
                    return 1

In [11]:
def turno_maquina(tablero_usuario, flota_usuario):
    while True:
        fila = random.randint(0, 9)
        columna = random.randint(0, 9)

        resultado = disparar((fila, columna), tablero_usuario)
        print(f"La máquina disparó a {(fila + 1, columna + 1)}: \n {resultado}")

        if resultado == "Agua":
            print("La máquina falló. Fin de su turno.")
            return 0  
        elif resultado == "Tocado":
            print("La máquina acertó. Sigue disparando.")
            for barco in flota_usuario:
                if barco_hundido(barco, tablero_usuario):
                    print("Barco hundido")
                    return 1

In [12]:
def disparar(casilla, tablero):
    fila, columna = casilla
    if tablero[fila, columna] == "O":
        tablero[fila, columna] = "X"
        return "Tocado"
    elif tablero[fila, columna] == '_':
        tablero[fila, columna] = "A"
        return "Agua"
    else:
        return "Casilla repetida"

In [13]:
def barco_hundido(barco, tablero):
    for i, j in barco:
        if tablero[i][j] != "X":
            return False
    return True

In [14]:
def mostrar_tablero(tablero):
    print("\n--- Tablero de la máquina ---")
    for fila in tablero:
        print(" ".join(fila))

In [15]:
def jugar():
    print("Welcome to Battleship")
    time.sleep(1)

    dibujar_batalla_barcos()
    time.sleep(2)

    modo = input("Hay dos modos de juego: Corto (cada jugador tendrá 6 barcos de eslora 1 y quien elimine antes dos barcos habrá ganado) o Largo (6 barcos cada uno y quien elimine los del contrincante antes habrá ganado). Introduzca: (Corto/Largo): ").lower()

    print("Creando tu nuevo tablero...")
    time.sleep(1)
    tablero_usuario = crear_tablero(10)
    tablero_maquina = crear_tablero(10)

    if modo == "corto":
        flota_usuario = crear_flota(tablero_usuario, [1] * 6)
        flota_maquina = crear_flota(tablero_maquina, [1] * 6)
    else:
        flota_usuario = crear_flota(tablero_usuario, [2, 2, 2, 3, 3, 4])
        flota_maquina = crear_flota(tablero_maquina, [2, 2, 2, 3, 3, 4])
        
    colocar_flota(flota_usuario, tablero_usuario)
    colocar_flota(flota_maquina, tablero_maquina)

    time.sleep(0.5)
    print("\nTu tablero")
    mostrar_tablero(tablero_usuario)

    while True:
        respuesta = input("¿Te gusta tu tablero o generamos otro? (si/no): ").lower()    
        if respuesta == "si":
            print("Entonces jugarás con este, ¡BUENA SUERTE!")
            break
        else:
            tablero_usuario = crear_tablero(10)
            flota_usuario = crear_flota(tablero_usuario, [2, 2, 2, 3, 3, 4])
            colocar_flota(flota_usuario, tablero_usuario)
            print("Nuevo tablero")
            mostrar_tablero(tablero_usuario)
               
    sistema_de_turnos(tablero_usuario, tablero_maquina, flota_usuario, flota_maquina, modo)

In [16]:
def hundir_la_flota():
    while True:
        jugar()
        jugar_nuevamente = input("¿Quieres jugar de nuevo? (si/no): ").lower()
        if jugar_nuevamente != "si":
            print("Saliendo del juego, que tenga buen día. No dude en volver a jugar.")
            break

